In [13]:
import numpy
# scipy.special for the sigmoid fintion expit()
import scipy.special
import matplotlib.pyplot
%matplotlib inline


# neural network class definition
class neuralNetwork :
    
    # initialise the neural network numpy.transpose(inputs))
    # 1.定义节点数量；2.随机初始权重；3.定义学习率；4.设置激活函数
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningreat) :
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # link weight martices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        '''使用正态概率分布采样权重，其中平均值为0，标准方差为节点传入链接数的开方'''
        self.wih = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))
        
        # learning rate
        self.lr = learningreat
        
        # activation function is the sigoid function
        # 匿名函数lambda接受了x，返回scipy.special.expit(x)，这就是S函数
        self.activation_function = lambda x: scipy.special.expit(x)
                                     
        pass
    
    def train(self, inputs_list, targets_list) :
        # 将输入列表转换为二维数组
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        #caulate signals into hidden/final_output layer
        #calculate tesignals emerging from hidden/final_output layer
        '''输入信号X_HIDDEN = W_input_hidden * I'''
        hidden_inputs = numpy.dot(self.wih, inputs) # 计算隐藏层中的输入信号
        hidden_outputs =self.activation_function(hidden_inputs) # 用激活函数计算隐藏层中出现的信号
        
        final_inputs = numpy.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors)
        
        # update the weight for the links between the hidden/input andoutput/hidden layers
        # 🔺W_j,k = a * E_k * O_k (1 - O_k) · (O_j).T
        # * 代表正常的对应元素的乘法；· 点乘是矩阵点积
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), 
                                       numpy.transpose(hidden_outputs))
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), 
                                       numpy.transpose(inputs))
        
        pass
    
    # query the neural network(给定输入，从输出节点给出答案)
    # 1.接受输入；2.将信号调整到下一层，并在下一层内调用S函数;3.返回最终输出结果
    def query(self, inputs_list) :
        # 将输入列表转换为二维数组
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        #caulate signals into hidden/final_output layer
        #calculate tesignals emerging from hidden/final_output layer
        '''输入信号X_HIDDEN = W_input_hidden * I'''
        hidden_inputs = numpy.dot(self.wih, inputs) # 计算隐藏层中的输入信号
        hidden_outputs =self.activation_function(hidden_inputs) # 用激活函数计算隐藏层中出现的信号
        
        final_inputs = numpy.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs
        

In [14]:
# number of input, hidden and output node
input_nodes = 784 # 为什么选择784个输入节点呢？这是28×28的结果，即图像像素个数
hidden_nodes = 200
output_nodes = 10

# learningrate is 0.1
learning_rate = 0.1

# create instance f neural network
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

## 一些改进：  
    1.调整学习率
    2.多次运行
    3.改变网络形状

In [15]:
# load the mnist training data CSV file into a list
training_data_file = open("G:\\desktop\\mnist_train .csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

# train the neural network

'''有些人把训练集执行一次称为一个世代。具有10个世代的训练，意味着使用整个训练数据运行程序10次。
   通过提供更多爬下坡的机会，有助于在梯度下降过程中进行权重更新'''
# epochs is the number of times the training data set is used for training
# epochs是训练数据集用于训练的次数
epochs = 5

for e in range(epochs) :
    # go through all records in the training data set
    # 浏览数据集中的所有记录
    for record in training_data_list :
        # split the record by the ',' commas
        all_values = record.split(',') # 根据逗号将数据进行拆分
        # scale and shift the inputs
        '''因为激活函数的缘故，输出值范围只能是（0，1），如果输入数据和输出值形状正好适合，
           就可以待在网络节点激活函数的舒适区内
               1.将输入颜色值从较大的0到255的范围，缩放至较小的0.01到1的范围
               2.将所得到的输入乘以0.99，把它们的范围变成0.0到0.99，再加上0.01，
                 将这些值整体偏移到所需的范围0.01到1.00'''
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        
        # create the target output values(all 0.01, except the desired label which is 0.99)
        '''正确的目标是：除了目标标签值为0.99，其余值为0.01'''
        targets = numpy.zeros(output_nodes) + 0.01 
        # all_value[0] is the target label for this record
        targets[int(all_values[0])] = 0.99 # 数据中第一个值是标签，即书写者实际希望表示的数字
        n.train(inputs,targets) # 调用训练函数，每一幅图片训练一次
        pass
    pass

In [16]:
# load the mnist test data CSV file into a list
test_data_file = open("G:\\desktop\\mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

# test the neural network

# scorecard for how well the network perform, initially empty
scorecard = []

# go through all the records in the test data set
for record in test_data_list :
    # split the record by the ',' commas
    all_values = record.split(',') # 用逗号拆分文本记录，分离出数值
    # correct answer is first value
    correct_label = int(all_values[0]) # 记下第一个数字，这是正确答案
    # scale and shift the inputs
    # 重新调整剩下的值，让他们适合用于查询神经网络
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    # query the network
    outputs = n.query(inputs) # 将来自神经网络的回答保存在outputs变量中
    
    '''numpy.argmax()返回最大值的索引，其中有一个参数axis，默认是0，表示哪一维的最大值索引。'''
    # the index of the highest value corresponds to the label
    label = numpy.argmax(outputs) # 返回最大值索引
    # append correct or incorrect to list
    if (label == correct_label) : # 将标签与已知的正确标签进行比较，如果他们是相同的，计分卡记“1”，否则记“0”
        # network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else :
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
    
    pass

In [17]:
# calculate the performance score, the fraction of correct answers
# 计算绩效得分，正确答案的分数
scorecard_array = numpy.asarray(scorecard) # 将列表转变为一维数组
print("performance = ", scorecard_array.sum() / scorecard_array.size)

performance =  0.9736
